In [1]:
%load_ext autoreload
%autoreload

import io
import os
import sys
import django
import glob
import json
import re
import requests
import pandas as pd
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers
from tqdm.notebook import tqdm
from PIL import Image, ImageSequence

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

import getch.models as m

In [2]:
endpoint = 'https://api.omnious.com/tagger/v2.12/tags'

headers = { 
    'x-api-key': 'z75BOQL2oFIepdufKGm8kqYtNnybCVHUZag3XPi6',
    'accept-language': 'ko'
}

def js(p):
    return {
        "image": {
            "type": "url",
            "content": p.src.url
        },
        "context": {
            "id": p.id,
            "detection": [
                "TOP",
                "BOTTOM",
                "WHOLEBODY",
                "SWIMWEAR",
                "SHOES",
                "BAG",
                "HAT",
                "JEWELRY",
                "ACCESSORY"
            ]
        }
    }

def get_value(tag, what):
    try:
        return tag[what]['name']
    except:
        try:
            return tag[what][0]['name']
        except:
            return None
        
        
def db_taggerize(pix):
    for _obj in pix.taggerlog['data']['objects']:
        for _tag in _obj['tags']:
            _type = _obj['type'].lower()
            _category = _tag['category']['name']
            _item = _tag['item']['name']
            _x = round(_tag['position']['x'], 3)
            _y = round(_tag['position']['y'], 3)
            
            if m.Tag.objects.filter(pix=pix, type=_type, category=_category, item=_item, x=_x, y=_y).exists():
                continue
            
            t = m.Tag(
                pix=pix,
                type=_type,
                category=_category,
                item=_item
            )

            t.x = _x
            t.y = _y
            t.color = get_value(_tag, 'colors')
            t.detail = get_value(_tag, 'details')
            t.pattern = get_value(_tag, 'prints')
            t.texture = get_value(_tag, 'textures')
            t.look = get_value(_tag, 'looks')
            t.length = get_value(_tag, 'length')
            t.sleeve_length = get_value(_tag, 'sleeveLength')
            t.neckline = get_value(_tag, 'neckLine')
            t.fit = get_value(_tag, 'fit')
            t.shape = get_value(_tag, 'shape')

            t.heel_height = get_value(_tag, 'heelHeight')
            t.heel_shape = get_value(_tag, 'heelShape')
            t.toe_type = get_value(_tag, 'toeType')
            t.sole_type = get_value(_tag, 'soleType')

            t.strap = get_value(_tag, 'strap') == '스트랩있음'
            t.size = get_value(_tag, 'size')
            t.main_material = get_value(_tag, 'mainMaterials')
            t.sub_material = get_value(_tag, 'subMaterials')

            t.save()

In [4]:
for _pix in tqdm(m.Pix.objects.order_by('-id')[1500:]):
    if (not _pix.tagger_called) or _pix.tagger_quota_limited:
        res = requests.post(endpoint, json=js(_pix), headers=headers)
        _pix.taggerlog = res.json()
        _pix.save()
        
    if _pix.tagged:
        db_taggerize(_pix)

  0%|          | 0/2798 [00:00<?, ?it/s]